In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from random_useragent.random_useragent import Randomize
from bs4 import BeautifulSoup
import random
import csv
import pandas as pd
from pynput.mouse import Button, Controller
import keyboard

import time
from pyautogui import press, typewrite, hotkey


In [2]:

def create_file_usernames():   #create our Data file #names_file.csv
    # field names
    fields = ['name','url']
    
    filename = "names_file.csv"
    
    # writing to csv file
    with open(filename, 'w') as csvfile:
        # creating a csv writer object
        csvwriter = csv.writer(csvfile)
    
        # writing the fields
        csvwriter.writerow(fields)





In [3]:
#create_file_usernames()

In [4]:



def Random_agent_generator():
    
    r_agent = Randomize()
    agent = r_agent.random_agent('desktop','windows') # returns 'Desktop / Linux'
    
    return agent 

def get_names_url(driver):
    
    #from a main okcupid page get the names and url
    #return the current name + url set to go for
    
    soup_level1=BeautifulSoup(driver.page_source, 'html.parser')
    
    #mydivs = soup_level1.find_all("a", "name")
    mydivs = soup_level1.find_all("a", "match-results-card")

    urlstore = [None] * len(mydivs)
    names = [None] * len(mydivs)
    
    #we store names
    for i in range(0,len(mydivs)):
        urlstore[i] = mydivs[i]['href']
    
    #we store urls
    for i in range(0,len(mydivs)):
        names[i] = mydivs[i].get_text()
    
    
    
    return names,urlstore

def addto(name,data,url):
    #example addto("names_file.csv",names,url)
    #names_file.csv
    for i in range(0,len(data)):
        with open(name, 'a', encoding = 'utf-8') as f:
            writer = csv.writer(f)
            writer.writerow([data[i],url[i]])



def move_mouse(): #move the mouse in the click area to prepare sending text
    
    mouse = Controller()
    mouse.position
    
    Xrandom = random.randint(700, 900)  #1050,1450)
    Yrandom = random.randint(620,630)
    
    mouse.position = (Xrandom, Yrandom)
    
    return mouse





In [5]:
#mode 0, send to the double take messages 
#mode 1, send to browse matchs the messages

def message_mode(mode):
        
        
    agent = Random_agent_generator()  #generate a random agent user for the scraping
    opts = Options()
    opts.add_argument("User-agent= %s" %agent)
    driver = webdriver.Chrome(executable_path='/home/j/chromedriver')    #(chrome_options=opts)
    driver.get("https://www.okcupid.com/")
    
    
    ### login on okcupid
    
    element = driver.find_element_by_xpath('//*[@id="root"]/span/div/div/div[1]/div[1]/div[2]/a')
    element.click()
    
    login = driver.find_element_by_name('username')
    login.send_keys("s@s")  #your email
    time.sleep(1)
    password = driver.find_element_by_name('password')
    password.send_keys("password")    #your password
    time.sleep(5)
    
    toclick = driver.find_element_by_xpath('//*[@id="root"]/span/div/div/div[2]/span/div/form/div[3]/input')
    toclick.click()
    
    time.sleep(5)
    #go on the browsing page
    
    toclick = driver.find_element_by_xpath('//*[@id="navigation"]/div[1]/ul/li[1]/a')
    toclick.click()
    
    
    
    
    if (mode == 0):
        
        for M in range(0,100):
        
            ###create a behavioral loop-
            #if we are on the main page we can add a couple of likes to our list (10)
            
            toclick = driver.find_element_by_xpath('//*[@id="navigation"]/div[1]/ul[1]/li[2]/a[1]/span[1]')
            toclick.click()
            
            try:
                for i in range(0,15):
                    toclick = driver.find_element_by_xpath('//*[@id="quickmatch-wrapper"]/div/div/span/div/div[2]/div/div[2]/span/div/div/div/div[1]/div[2]/button[2]')
                    toclick.click()
                    time.sleep(2)
            except:
                print('you liked each other, move next one')
            
            
            #  we want to send a message to those new likes
                
            driver.get("https://www.okcupid.com/who-you-like")
            driver.execute_script("window.scrollTo(0, 9000);")
            time.sleep(2)
            driver.execute_script("window.scrollTo(0, 4000);")
            time.sleep(2)
            
            soup_level1=BeautifulSoup(driver.page_source, 'html.parser')
            
            mydivs = soup_level1.find_all("a", { "class" : "userrow-inner"})
            
            urlstoreX = [None] * len(mydivs)
            #we store urls
            for i in range(0,len(mydivs)):
                urlstoreX[i] = mydivs[i]['href']
            
            
            namess =[None]*len(urlstoreX)
            for i in range(0,len(mydivs)):      #get the text of the twits
                namess[i] = (mydivs[i].get_text()).partition(' ')[0][:len((mydivs[i].get_text()).partition(' ')[0])-2]
            
            
            namestogo = []
            urltogo = []
            exception1 = 0
            exception2 = 0
            
            load = pd.read_csv("names_file.csv")
            
            for i in range(0,len(mydivs)):
                for j in range(len(load['name'])):
                    if(namess[i] == load['name'][j] ):
                        exception1 = 1 #raised exception for a specific i
                    if( urlstoreX[i] == load['url'][j]):
                        exception2 = 1    
                if (exception1 == 1 and exception2 == 1):
                    blue = 0
                    #print('we skipped %s already in database' %names[i])
                else:
                    urltogo.append(urlstoreX[i]) #we store the url we have to go
                    namestogo.append(namess[i])
            
            
            for i in range(0,len(urltogo)):     #we loop every profile we have to go
                
            
                try:
                    
                    driver.execute_script("window.open('');")
                    
                    driver.switch_to.window(driver.window_handles[1])
                    driver.get("https://www.okcupid.com%s" %urltogo[i])
                    
                    soup_level=BeautifulSoup(driver.page_source, 'html.parser')
                    random_text = soup_level.find_all("p", "profile-essay-contents")
                    
                    #we store urls
                    
                    try: 
                        for j in range(0,len(random_text)):
                            random_text[j] = random_text[j].get_text()
                        randomparagraph_num = random.randint(0, len(random_text)-1)
                        randomparagraph = random_text[randomparagraph_num]
                        if (len(randomparagraph) > 50):
                            randomparagraph = randomparagraph[0:50]
                                
                    except: 
                        randomparagraph = 'you have no information on your profile'
                    
                    her_name = str(namestogo[i])
                    #message = "Hi,"  + aa + "  :)" + "  I really liked the 'red lipstick', nice touch"
                    message = "Hi, " + her_name + ", I saw you wrote :" 
                    message = message + str(randomparagraph) + ", can you tell me more about that? That sounds nice."
                    #message ="Hi"
                    try:   
                        toclick = driver.find_element_by_xpath('//*[@id="like-button"]')
                        toclick.click()   
                        time.sleep(1)
                          
                        try :
                            toclick = driver.find_element_by_xpath('//*[@id="like-explainer"]/div[1]/div[3]/ul/li/button')
                            toclick.click()  
                            time.sleep(1)
                        except: 
                            print('all good,')
                        
                    except: 
                        
                        print('error, move next one')
                        toclick = driver.find_element_by_xpath('//*[@id="profile_actions"]/div/div/span/div/div/button[2]')
                        toclick.click()  
                        time.sleep(1)
                    
                
                    time.sleep(1)
                    
                    time.sleep(1)
                  
                    time.sleep(1)
                    #move_mouse()
                    
                    toclick = driver.find_element_by_xpath("//*[@id='global_messaging_V2']/div[@class='messenger']/div[@class='messenger-main-window messenger-main-window--open']/div[@class='messenger-main-window-container']/div[@class='messenger-composer-wrapper']/textarea[@class='messenger-composer']")
                    toclick.click()  
                    time.sleep(1)
                    
                    keyboard.write(message)
            
                    toclick = driver.find_element_by_xpath('//*[@id="global_messaging_container"]/div/form/button')
                    toclick.click()  
                    time.sleep(1)
                    
                    #quit the message dialog
                    toclick = driver.find_element_by_xpath('//*[@id="global_messaging_container"]/div[1]/div[1]/div[1]/button[3]/i[1]')
                    toclick.click()  
                    time.sleep(1)
                    
                    #reset go back to main page
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                
                except: # a random bug occured we need to reset at the next i of the for loop
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    continue
            
            addto("names_file.csv",namestogo,urltogo) #we store the n😝😝ew names and url that we didnt knew about
                
            driver.get("https://www.okcupid.com")
            print('end of loop ')
            time.sleep(2)
            
    
    
    
    if (mode == 1):
            
        for M in range(0,100):
        
            names, url = get_names_url(driver)
            load = pd.read_csv("names_file.csv")
            
            namestogo = []
            urltogo = []
            exception1 = 0
            exception2 = 0
            
            for i in range(0,len(names)):
                for j in range(len(load['name'])):
                    if(names[i] == load['name'][j] ):
                        exception1 = 1 #raised exception for a specific i
                    if( url[i] == load['url'][j]):
                        exception2 = 1    
                if (exception1 == 1 and exception2 == 1):
                    blue = 0
                    #print('we skipped %s already in database' %names[i])
                else:
                    urltogo.append(url[i]) #we store the url we have to go
                    namestogo.append(names[i])
               
            
            for i in range(0,len(urltogo)):     #we loop every profile we have to go
                
            
                try:
                    
                    driver.execute_script("window.open('');")
                    
                    driver.switch_to.window(driver.window_handles[1])
                    driver.get("https://www.okcupid.com%s" %urltogo[i])
                    
                    soup_level=BeautifulSoup(driver.page_source, 'html.parser')
                    random_text = soup_level.find_all("p", "profile-essay-contents")
                
                    #we store urls
                    
                    try: 
                        for j in range(0,len(random_text)):
                            random_text[j] = random_text[j].get_text()
                        randomparagraph_num = random.randint(0, len(random_text)-1)
                        randomparagraph = random_text[randomparagraph_num]
                        if (len(randomparagraph) > 50):
                            randomparagraph = randomparagraph[0:50]
                            
                    except: 
                        randomparagraph = 'you have no information on your profile'
                    
                    her_name = str(namestogo[i])
                    her_name = namestogo[i].split(",")[0]
                    
#                     message = "Hi,"  + aa + "  :)" + "  I really liked the 'red lipstick', nice touch"
#                     message = "Hi, " + her_name + ", I saw you wrote :" 
#                     message = message + str(randomparagraph) + ", can you tell me more about that? That sounds nice."
                    #message = "Hi "
                    try:   
                        toclick = driver.find_element_by_xpath('//*[@id="like-button"]')
                        toclick.click()   
                        time.sleep(1)
                          
                        try :
                            toclick = driver.find_element_by_xpath('//*[@id="like-explainer"]/div[1]/div[3]/ul/li/button')
                            toclick.click()  
                            time.sleep(1)
                        except: 
                            print('all good,')
                        
                    except: 
                        
                        print('error, move next one')
                        toclick = driver.find_element_by_xpath('//*[@id="profile_actions"]/div/div/span/div/div/button[2]')
                        toclick.click()  
                        time.sleep(1)
                    
                
                    time.sleep(1)
                    #move_mouse()
                    time.sleep(1)
                    #move_mouse()
                    time.sleep(1)
                    #move_mouse()
                    
#                     mouse = Controller()
#                     mouse.click(Button.left,1)
                    #print ("i am going to type")
                    
                    toclick = driver.find_element_by_xpath("//*[@id='global_messaging_V2']/div[@class='messenger']/div[@class='messenger-main-window messenger-main-window--open']/div[@class='messenger-main-window-container']/div[@class='messenger-composer-wrapper']/textarea[@class='messenger-composer']")
                    toclick.click()  
            
                    time.sleep(1)
                    #keyboard.write("Hi")
                    time.sleep(1)
                    #press("hi")
                    time.sleep(1)
                    #typewrite("Hi, hope you are having a plesant day", interval=.50)
                    #typewrite("Hi " + her_name + ", really liked what you wrote. If you are around let us converse to see how things go. Good day!", interval=.1)
                    typewrite("Hi " + her_name + 
                              ", Happy Christmas. Jesus taught us hope and love. May we get the love of our life in 2019... Hoping to hear back from you!!", interval=.1)

                    time.sleep(2)
                    toclick = driver.find_element_by_xpath("//*[@id='global_messaging_V2']/div[@class='messenger']/div[@class='messenger-main-window messenger-main-window--open']/div[@class='messenger-main-window-container']/div[@class='messenger-toolbar']/button[@class='messenger-toolbar-send']/span")
   #div[@id="global_messaging_V2'""/div[@class="messenger"]/div[@class="messenger-main-window messenger-main-window--open"]/div[@class="messenger-main-window-container"]/div[@class='messenger-toolbar']/button
            
                    #mouse.position = (955, 658)
                    #mouse = Controller()
                    #mouse.click(Button.left,1)
                    
                    #toclick = driver.find_element_by_xpath('//*[@id="global_messaging_V2"]/div/form/button')
#x:955 y:658
                    toclick.click()  
                    time.sleep(3)
                    
                    #quit the message dialog
                    toclick = driver.find_element_by_xpath('//*[@id="global_messaging_container"]/div[1]/div[1]/div[1]/button[3]/i[1]')
                    toclick.click()  
                    time.sleep(1)
                    
                    #reset go back to main page
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                
                except: # a random bug occured we need to reset at the next i of the for loop
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    continue
        
           
            addto("names_file.csv",namestogo,urltogo) #we store the n😝😝ew names and url that we didnt knew about
                
            driver.get("https://www.okcupid.com/match")
            print('end of loop 1')
            time.sleep(2)
            
            driver.execute_script("window.scrollTo(0, 9000);")
            time.sleep(2)
            driver.execute_script("window.scrollTo(0, -4000);")
            time.sleep(2)
            driver.execute_script("window.scrollTo(0, 9000);")
            time.sleep(1)
            driver.execute_script("window.scrollTo(0, 4000);")
            


In [ ]:
message_mode(1)

error, move next one
error, move next one
all good,
all good,
all good,
all good,
all good,
error, move next one
all good,
error, move next one
error, move next one
end of loop 1
error, move next one
all good,
all good,
error, move next one
all good,
all good,
error, move next one
error, move next one
end of loop 1
all good,
all good,
all good,
error, move next one
all good,
all good,
all good,
all good,
error, move next one
all good,
error, move next one
all good,
all good,
error, move next one
error, move next one
all good,
all good,
all good,
all good,
error, move next one
all good,
all good,


In [23]:
press("hi")H

In [ ]:
import keyboard
OUTPUT = "None"
def write(string):
    global OUTPUT
    OUTPUT = string
    keyboard.write(string)
keyboard.add_hotkey("Ctrl+Alt+k", write, ["hello world"])
input()
print(OUTPUT)

In [ ]:
#keyboard.write("hi")
import keyboard
keyboard.hook(print)


In [ ]:

def get_names_url(driver):
    
    #from a main okcupid page get the names and url
    #return the current name + url set to go for
    
    soup_level1=BeautifulSoup(driver.page_source, 'html.parser')
    
    mydivs = soup_level1.find_all("a", "name")
    urlstore = [None] * len(mydivs)
    names = [None] * len(mydivs)
    
    #we store names
    for i in range(0,len(mydivs)):
        urlstore[i] = mydivs[i]['href']
    
    #we store urls
    for i in range(0,len(mydivs)):
        names[i] = mydivs[i].get_text()
    
    
    
    return names,urlstore

In [39]:
from pyautogui import press, typewrite, hotkey

from time import sleep # this should go at the top of the file

sleep(5)
driver = webdriver.Chrome(executable_path='/home/j/chromedriver')    #(chrome_options=opts)
driver.get("https://www.okcupid.com/")

### login on okcupid
    
element = driver.find_element_by_xpath('//*[@id="root"]/span/div/div/div[1]/div[1]/div[2]/a')
element.click()

login = driver.find_element_by_name('username')
login.send_keys("shivgan32587@qcfinance.in")  #your email
time.sleep(1)
password = driver.find_element_by_name('password')
password.send_keys("namahshivay")    #your password
time.sleep(1)

toclick = driver.find_element_by_xpath('//*[@id="root"]/span/div/div/div[2]/span/div/form/div[3]/input')
toclick.click()

time.sleep(2)
#go on the browsing page



toclick = driver.find_element_by_xpath('//*[@id="navigation"]/div[1]/ul/li[1]/a')
toclick.click()

names, url = get_names_url(driver)
urltogo=url

driver.execute_script("window.open('');")

                    
driver.switch_to.window(driver.window_handles[1])
driver.get("https://www.okcupid.com%s" %urltogo[0])

soup_level=BeautifulSoup(driver.page_source, 'html.parser')
random_text = soup_level.find_all("p", "profile-essay-contents")


try:   
        toclick = driver.find_element_by_xpath('//*[@id="like-button"]')
        toclick.click()   
        time.sleep(1)

        try :
            toclick = driver.find_element_by_xpath('//*[@id="like-explainer"]/div[1]/div[3]/ul/li/button')
            toclick.click()  
            time.sleep(1)
        except: 
            print('all good,')

except: 

    print('error, move next one')
    toclick = driver.find_element_by_xpath('//*[@id="profile_actions"]/div/div/span/div/div/button[2]')
    toclick.click()  
    time.sleep(1)


time.sleep(1)
move_mouse()
time.sleep(1)
move_mouse()
time.sleep(1)
move_mouse()

mouse = Controller()
mouse.click(Button.left,1)
#print ("i am going to type")

time.sleep(1)
#keyboard.write("Hi")


#typewrite("Hi, hope you are having a plesant day", interval=.50)
typewrite("Hi there, really liked what you wrote. If you are around let us converse to see how things go. Good day!", interval=.01)
#time.sleep(9)
#toclick = driver.find_element_by_xpath('//*div[@id="global_messaging_V2/div[@class="messenger"]/div[@class="messenger-main-window messenger-main-window--open"]/div[@class="messenger-main-window-container"]/div[@class="messenger-toolbar"]/button')
#div[@id="global_messaging_V2'""/div[@class="messenger"]/div[@class="messenger-main-window messenger-main-window--open"]/div[@class="messenger-main-window-container"]/div[@class='messenger-toolbar']/button

#mouse.position = (955, 658)
#mouse = Controller()
#mouse.click(Button.left,1)

#toclick = driver.find_element_by_xpath('//*[@id="global_messaging_V2"]/div/form/button')
#toclick = driver.find_element_by_xpath("div[@class='messenger-toolbar']/button[@class='messenger-toolbar-send']")
toclick = driver.find_element_by_xpath("//*[@id='global_messaging_V2']/div[@class='messenger']/div[@class='messenger-main-window messenger-main-window--open']/div[@class='messenger-main-window-container']/div[@class='messenger-toolbar']/button[@class='messenger-toolbar-send']/span")



#div[@id='global_messaging_V2']/div[0]/div[0]/div[0]/div[0]/button
#x:955 y:658
toclick.click()  
time.sleep(5)
                    

html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
print (html)

error, move next one
<head><script src="https://pagead2.googlesyndication.com/pagead/osd.js"></script><script type="text/javascript" async="" src="//static.criteo.net/js/ld/publishertag.prebid.js"></script> <meta charset="UTF-8"> <meta http-equiv="content-type" content="text/html; charset=utf-8"> <link rel="dns-prefetch" href="//includes.okccdn.com"> <link rel="dns-prefetch" href="//cdn.okccdn.com"> <link rel="preconnect" href="//includes.okccdn.com"> <link rel="preconnect" href="//cdn.okccdn.com">  <link rel="preload" as="font" href="//includes.okccdn.com/flat/fonts/GT-America/Regular/GT-America-Regular.woff" type="font/woff2" crossorigin="">  <link rel="preload" as="font" href="//includes.okccdn.com/flat/fonts/GT-America/Medium/GT-America-Medium.woff" type="font/woff2" crossorigin="">  <link rel="preload" as="font" href="//includes.okccdn.com/flat/fonts/GT-America/Black/GT-America-Black.woff" type="font/woff2" crossorigin="">     <title>Ph / 30 / Clifton, NJ | OkCupid</title> <link r

In [35]:
toclick = driver.find_element_by_xpath('//*[@id="global_messaging_V2"]/div/div/div/div/button')
#div[@id='global_messaging_V2']/div[0]/div[0]/div[0]/div[0]/button
#x:955 y:658
toclick.click()  

In [45]:
names[0].split(",")[0]

'Lo'